## RNASeq Development

In [1]:
import os
import subprocess
import re
import logging
from collections import defaultdict
from datetime import datetime
import tempfile
import pickle
from glob import glob

In [2]:
from importlib import reload
import dogma.dna.define
import dogma.utils
reload(dogma.dna.define)
from dogma.dna.define import Region, Sequence, GeneSpace, Transcript
reload(dogma.utils)
from dogma.utils import ngsUtils, configUtils
#reload(ngsUtils)


In [3]:
import pandas as pd
import numpy as np
import pybedtools
from pybedtools import BedTool
import pysam

## Major Steps in the Pipeline
- Prepare Inputs / Configurations
    - Samples with BAM path 
    - Gene name
    - Output directory
- Generate bwa index for each isoform of a given gene
- Subset reads from BAM file(s) for a given gene region
    - Determine the genome build of BAM files to define region to subset reads
    - Use Sambamba/BedTools to extract paired end FASTQ reads in a region
    - Assign sample name to each FASTQ read
- Generate batches of FASTQ reads 
- Process each batch in the following way
    - Align paired-end FASTQ reads against all the isoforms of a gene using bwa aln
    - Parse BAM files to assign each read to one of the isoforms and mark for suitable variant call i.e. SNV/InDel/SV/ITD etc.
- Organize parsed output in sample- and isoform-wise manner
- Variant calling
- Annotations w.r.t. assigned isoforms 

## Functions

In [4]:
def prepareSampleNameMap(sampleFile: os.path, wdir: os.path, projectid: str) -> pd.DataFrame:
    """Prepare sample map to hold short SIDs"""
    sampleDf = pd.read_csv(sampleFile, sep="\t", header=None, names=['Sample', 'BAM'])
    sampleDf['SID'] = [f'S{i}' for i, _ in enumerate(sampleDf['Sample'], 1)]
    ofn = os.path.join(wdir, projectid + '_SampleMap.tsv')
    sampleDf.to_csv(ofn, sep="\t", index=False)
    return sampleDf

In [5]:
def prepareSampleConfig(sampleName: str, bamPath: os.path) -> dict:
    sampleConfig = {}
    sampleConfig['SID'] = sampleName
    sampleConfig['BAM'] = os.path.abspath(bamPath)
    sampleConfig['Genome'] = ngsUtils.aa06_determine_genome_build_of_bam(bamPath)
    chrTag = ngsUtils.aa05_does_bam_file_have_chr_tag(bamPath)
    if not chrTag:
        sampleConfig['removeChrPrefix'] = True
    else:
        sampleConfig['removeChrPrefix'] = False
    return sampleConfig

In [6]:
def extractGeneReadsFromBAM(bam: os.path, sampleName: str, outDir: os.path, geneConfigs:configUtils.GeneConfig, suffix='') -> list:
    """
    Extract paired end FASTQ reads from the BAM file for a given gene specified in geneConfigs
    """
    genome = ngsUtils.aa06_determine_genome_build_of_bam(bam)
    chrTag = ngsUtils.aa05_does_bam_file_have_chr_tag(bam)
    geneBedFname = os.path.join(outDir, geneConfigs.gene + f".gene.{genome}.bed")
    geneBamFname = os.path.join(outDir, geneConfigs.project + '_' + geneConfigs.gene + f"_{sampleName}_{suffix}.gene.{genome}")
    if not chrTag:
        bedFile = Region.convertRegionListToBed(geneConfigs.confDict[genome]['GeneRegions'], ofn=geneBedFname, removeChrPrefix=True)
    else:
        bedFile = Region.convertRegionListToBed(geneConfigs.confDict[genome]['GeneRegions'], ofn=geneBedFname, removeChrPrefix=False)
    #fastqFiles = ngsUtils.ba03_slice_bam_2_fastq(bam, bedFile, geneBamFname)
    fastqFiles = ngsUtils.ba14_slice_bam_2_fastq_withSampleHeader(bam, bedFile, geneBamFname, sampleName)
    return fastqFiles

In [7]:
def extractReadnameRecordFromBAM(
        readname: str, bam: os.path,
        sambambaPath: os.path
):
    extractCmd = f"{sambambaPath} view -F \" read_name == '{readname}' \" {bam}"
    process = subprocess.run([extractCmd], capture_output=True, text=True, shell=True)
    #print(process.returncode)
    columns=['qname', 'flag', 'rname', 'pos', 'mapq', 'cigar', 'rnext', 'pnext', 'tlen', 'seq', 'qual']
    if process.returncode == 0 and process.stdout != '':
        return pd.DataFrame(
            [record.split("\t")[0:11] for record in process.stdout.split("\n")[:-1]],
            columns=columns
        )
    else:
        return pd.DataFrame(columns=columns)

In [8]:
def extractReadnameRecordFromIsoformBAMs(
        readname: str, bamList: list, geneConfigs: configUtils.GeneConfig
):
    #def mergeBatchBamFiles(bamBatchDict: dict, geneConfigs: configUtils.GeneConfig):
    """
    Extract BAM records for given readname from all the isoforms in a given list of BAM file
    readname: read ID
    bamList: Lists of BAM files for all isoforms in a given batch.
    """
    readMappingDfs = []
    for bam in bamList:
        isoformDf = extractReadnameRecordFromBAM(readname, bam=bam, sambambaPath=geneConfigs.apps['sambamba'])
        readMappingDfs.append(isoformDf)
    return pd.concat(readMappingDfs).reset_index(drop=True)

In [9]:
def postProcessReadMappingDf(readIsoformMappingDf:pd.DataFrame):
    pass

In [10]:
def assignReadIsoform(readIsoformMappingDf: pd.DataFrame):
    """
    Assign read to a specific isoform
    readIsoformMappingDf: A dataframe of BAM records from gene isoforms for a single read id. 
    """
    pass

In [11]:
def concatFastqBatches(outdir, r1Pattern: str, r2Pattern: str, fq1: str, fq2: str) -> list:
    cmd1 = ["find", outdir, "-type f -name", r1Pattern, "-exec cat {} + >", fq1]
    cmd2 = ["find", outdir, "-type f -name", r2Pattern, "-exec cat {} + >", fq2]
    #find /dir/to/search/ -type f -name "FILE-TO-FIND-Regex" -exec rm {} \;
    # find . -type f -name "TARGETAML_NRAS*prebatch_1.gene.*.R1.fastq.gz" -exec cat {} + > file1.gz

    # find filename -name '*.txt' -print0 | sort -z | xargs -0 cat  > combined.txt
    cmd1 = ["find", outdir, "-type f -name", r1Pattern, "-print0 | sort -z | xargs -0 cat >", fq1]
    cmd2 = ["find", outdir, "-type f -name", r2Pattern, "-print0 | sort -z | xargs -0 cat >", fq2]

    cmd3 = ["find", outdir, "-type f -name", r1Pattern, "-exec rm {} \;"]
    cmd4 = ["find", outdir, "-type f -name", r2Pattern, "-exec rm {} \;"]

    os.system(" ".join(cmd1))
    os.system(" ".join(cmd2))
    os.system(" ".join(cmd3))
    os.system(" ".join(cmd4))

    return [fq1, fq2]

In [12]:
def extractReadNamesFromFASTQ(r1Fqs: list, geneConfigs: configUtils.GeneConfig):
    """
    Extract readnames per FASTQ batch
    """
    getReadsIDsJobDict = {}
    batchIDs = []
    readNameFiles = []
    seqkit = geneConfigs.apps['seqkit']
    for fq1 in r1Fqs:
        batch = os.path.basename(fq1).split(".")[-3]
        batchIDs.append(batch)
        jobNameRead = f'{geneConfigs.project}_{geneConfigs.gene}_{batch}_readnames'
        outReadFn = os.path.join(geneConfigs.WDIR, jobNameRead)
        readNameFiles.append(outReadFn)
        extractReadIn = os.path.join(geneConfigs.WDIR, f'{geneConfigs.project}_{geneConfigs.gene}_raw.R1.{batch}.fastq.gz')
        readExtractCmd = f'"{seqkit} fx2tab -n {extractReadIn} > {outReadFn}"'
        getReadsIDsJobDict[jobNameRead] = readExtractCmd
    readNamejobStatus = ngsUtils.submitAndMonitorJobs(
        getReadsIDsJobDict, wdir=geneConfigs.WDIR, jobDescription="Extract readnames from the FASTQ batches"
    )
    if readNamejobStatus[0]:
        return readNameFiles, batchIDs
    else:
        return False, False

In [13]:
def mapFastqReadsSinglePair(
        bwaIndex: os.path,
        fq1: os.path,
        fq2: os.path,
        outdir: os.path,
        prefix: str,
        threads: int = 4,

):
    mappingScript = "/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/rnaseq/rnaseq/mapReads.py"
    mappingJobDict = {}
    mapCmd = f"python {mappingScript} -index {bwaIndex} -fq1 {fq1} -fq2 {fq2} -prefix {prefix} -threads {threads}"
    mappingJobDict[prefix] = mapCmd

    mapJobStatus = ngsUtils.submitAndMonitorJobs(mappingJobDict, wdir=outdir, jobDescription=f"Map reads against {bwaIndex}")
    if mapJobStatus[0]:
          return os.path.join(outdir, prefix + ".bam")
    else:
      return False

In [14]:
def subsetFastqByReadNames(
        readsToExtract: os.path, 
        batchFq1: os.path, batchFq2: os.path, outSuffix: str,
        outdir: os.path
):
    r1File = os.path.join(outdir, outSuffix + "_rest.R1.fastq.gz")
    r2File = os.path.join(outdir, outSuffix + "_rest.R2.fastq.gz")
    seqkit = "/research/rgs01/applications/hpcf/authorized_apps/rhel7_apps/seqkit/vendor/2.1.0/seqkit"
    extractFqCmd1 = f"{seqkit} grep -n -f {readsToExtract} -o {r1File} {batchFq1}"
    r1Pstatus = subprocess.call([extractFqCmd1], shell=True)
    extractFqCmd2 = f"{seqkit} grep -n -f {readsToExtract} -o {r2File} {batchFq2}"
    r2Pstatus = subprocess.call([extractFqCmd2], shell=True)
    if (r1Pstatus + r2Pstatus) == 0:
        return r1File, r2File
    else:
        print("Something went wrong while extracting unmatched reads from the FASTQ file")
        return False, False


In [15]:
def parsePerfectMappedReads(
        bam: os.path, outSuffix: str, outdir: os.path, mappingQuality: int = 55
):
    outBamfile = os.path.join(outdir, outSuffix + '.bam')
    extractMatchedReadsBamCmd = f'sambamba view -f bam -o {outBamfile} -F " mapping_quality >= {mappingQuality} and cigar =~ /^\d+M$/ " {bam} '
    print(extractMatchedReadsBamCmd)
    extractMatchedReadsBamCmdStatus = subprocess.call([extractMatchedReadsBamCmd], shell=True)
    
    if extractMatchedReadsBamCmdStatus == 0:
        unMatchedRestReadsFile = os.path.join(outdir, outSuffix + "_UNMATCHED_READNAMES")
        extractUnMatchedReadIDsCmd = f'sambamba view -F " not ( mapping_quality >= {mappingQuality} and cigar =~ /^\d+M$/ ) " {bam} | cut -f1 | sort | uniq > {unMatchedRestReadsFile}'
        extractUnMatchedReadIDsCmdStatus = subprocess.call([extractUnMatchedReadIDsCmd], shell=True)
        if extractUnMatchedReadIDsCmdStatus == 0:
            extractMatchedSingleReadIDsCmd_p1 = f'sambamba view -F " not ( mapping_quality >= {mappingQuality} and cigar =~ /^\d+M$/ ) " {bam} | cut -f1 | sort | uniq -c '
            extractMatchedSingleReadIDsCmd_p2 = f" | awk -F '[[:space:]]+' '{{if ( $2 == 1 ) print $3}}' >> {unMatchedRestReadsFile}"
            extractMatchedSingleReadIDsCmdStatus = subprocess.call([extractMatchedSingleReadIDsCmd_p1, extractMatchedSingleReadIDsCmd_p2], shell=True)
            if extractMatchedSingleReadIDsCmdStatus == 0:
                return outBamfile, unMatchedRestReadsFile
        else:
            print(f"Something went wrong while parsing perfectly matched reads from {bam}")
    else:
        return False, False

In [16]:
def processFastqBatchForIsoforms(batchFq1, batchFq2, orderedIsoforms, geneConfigs):
    readsToExtract = None
    iterationNum = 0
    batchIsoBams = []
    batchID = os.path.basename(batchFq1).split(".")[-3]
    for isoform in orderedIsoforms:
        print(f"Iteration: {iterationNum}")
        isoformName = "_".join(os.path.basename(isoform).replace(".fa", "").split("_")[2:])
        outSuffix = f"{geneConfigs.project}_{geneConfigs.gene}_{isoformName}_{batchID}"
        if (readsToExtract is None) and (iterationNum == 0):
            fq1 = batchFq1
            fq2 = batchFq2
            
        else:
            print("In subsetting FASTQ step")
            fq1, fq2 = subsetFastqByReadNames(readsToExtract, batchFq1, batchFq2, outSuffix, outdir=geneConfigs.WDIR)
        print("Mapping reads")
        isoformBam = mapFastqReadsSinglePair(
            bwaIndex=isoform, fq1=fq1, fq2=fq2, outdir= geneConfigs.WDIR,
            prefix=outSuffix
        )
        print(isoformBam)
        outSuffix = outSuffix + f"_MATCHEDREADS_{iterationNum}"
        if isoformBam:
            print(isoformBam, outSuffix)
            outBamfile, unMatchedRestReadsFile = parsePerfectMappedReads(isoformBam, outdir=geneConfigs.WDIR, outSuffix=outSuffix)
            batchIsoBams.append(outBamfile)
            readsToExtract = unMatchedRestReadsFile
        iterationNum += 1            
    return batchIsoBams

In [17]:
def batchWisePerfectMatchedBAMs(
        splitFq1: list, splitFq2: list, geneConfigs: configUtils.GeneConfig
):
    """
    Submit parallel jobs to generate perfectly matched reads BAM files per isoform per FASTQ batch
    """
    batchJobsDict = {}
    matchedBams = False
    completeBams = False
    pickleFile = geneConfigs.configPickle
    mapQ = geneConfigs.confDict['PARAMETERS']['mapq']
    genome = geneConfigs.confDict['PARAMETERS']['mapgenome']
    for batch in range(len(splitFq1)):
        scriptName = "/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/rnaseq/rnaseq/mapReadsInIsoformOrder.py"
        cmd = f"python {scriptName} -fq1 {splitFq1[batch]} -fq2 {splitFq2[batch]} -genome {genome} -pickelfile {pickleFile} -mapQ {mapQ}"
        jobName = f"Processing_FASTQ_Batch_{batch + 1}_for_all_Isforms"
        batchJobsDict[jobName] = cmd
    batchJobStatus = ngsUtils.submitAndMonitorJobs(
        jobDict=batchJobsDict, wdir=geneConfigs.WDIR, 
        jobDescription="Map FASTQ Batches in order of isoforms to subset perfectly matched reads for counting"
    )

    if batchJobStatus[0]:

        matchedBams = {}
        completeBams = {}
        for fq in splitFq1:
            i = 0
            batchID = os.path.basename(fq).split(".")[-3]
            for bwaIndex in geneConfigs.confDict[genome]['bwaIndices']:
                isoformName = "_".join(os.path.basename(bwaIndex).replace(".fa", "").split("_")[2:])
                outSuffixCompleteBam = f"{geneConfigs.project}_{geneConfigs.gene}_{isoformName}_{batchID}.bam"
                if isoformName not in list(matchedBams.keys()):
                    matchedBams[isoformName] = []
                if isoformName not in list(completeBams.keys()):
                    completeBams[isoformName] = []
                outSuffixMatched = f"{geneConfigs.project}_{geneConfigs.gene}_{isoformName}_{batchID}_MATCHEDREADS_{i}.bam"
                matchedBams[isoformName].append(os.path.join(f"{geneConfigs.WDIR}", outSuffixMatched))
                completeBams[isoformName].append(os.path.join(f"{geneConfigs.WDIR}", outSuffixCompleteBam))
                i += 1

    return batchJobStatus, completeBams, matchedBams
    

In [18]:
def alleleCountingForPerfectMatchedBAMs(
        matchedBams: dict, geneConfigs: configUtils.GeneConfig
):
    """
    Submit parallel jobs for allele counting in perfectly matched BAMs per isoform batch
    """
    countJobsDict = {}
    countDict = {}
    errDict = {}
    pickleFile = geneConfigs.configPickle
    qcut = geneConfigs.confDict['PARAMETERS']['qcut'] 
    mapq = geneConfigs.confDict['PARAMETERS']['mapq']
    trimlen = geneConfigs.confDict['PARAMETERS']['trimlen']
    minReadBQ = geneConfigs.confDict['PARAMETERS']['minreadbq']
    fcut = geneConfigs.confDict['PARAMETERS']['minreadfcut']
    minReadDepth = geneConfigs.confDict['PARAMETERS']['mindepth']
    sampleNameInReads = geneConfigs.confDict['PARAMETERS']['samplenameinreads']
    
    for isoform in matchedBams.keys():
        for bam in matchedBams[isoform]:
            scriptName = "/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/rnaseq/rnaseq/alleleCounting.py"
            jobName = os.path.basename(bam).replace(".bam", ".count")
            ofn = os.path.join(geneConfigs.WDIR, jobName)
            cmd = f"python {scriptName} -bam {bam} -ofn {ofn} -qcut {qcut} -mapq {mapq} -trimlen {trimlen} -minReadBQ {minReadBQ} -fcut {fcut} -minReadDepth 0 -sampleNameInReads {sampleNameInReads}"
            countJobsDict[jobName] = cmd

    countJobStatus = ngsUtils.submitAndMonitorJobs(
        jobDict=countJobsDict, wdir=geneConfigs.WDIR, 
        jobDescription="Counting alleles in perfectly matched BAMs in batch-wise manner"
    )

    if countJobStatus[0]:

        countDict = {}
        errDict = {}
        for isoform in matchedBams.keys():
            for bam in matchedBams[isoform]:
                countfile = os.path.join(geneConfigs.WDIR, os.path.basename(bam).replace(".bam", ".count"))
                errfile = countfile + '_err'
                
                if isoform not in list(countDict.keys()):
                    countDict[isoform] = [countfile]
                    errDict[isoform] = [errfile]
                else:
                    countDict[isoform].append(countfile)
                    errDict[isoform].append(errfile)

    return countJobStatus, countDict, errDict
    

In [19]:
def splitFastq(
        fq1: os.path, fq2: os.path, 
        batchSize: int,
        outdir: os.path,
        jobName: str,
        seqkit: os.path,
        threads: int = 1,
        local: bool = False
):
    splitFq1 = []
    splitFq2 = []
    splitCommand = f"{seqkit} split2 -s {batchSize} -1 {fq1} -2 {fq2} -j {threads} -O {outdir}"
    if local:
        os.system(splitCommand)
        return jobName
    else:
        jobDict = {jobName : splitCommand}
        splitJobDict = ngsUtils.submitJobs(jobDict, wdir=outdir)
        print(f"Submitted a job to split FASTQ file")
        splitJobStatus = ngsUtils.aa08_job_manager_succeeded(
            splitJobDict, only_check_finish=False, initial_sleep_sec=10, wait_sec=5, outpath=outdir
        )
        if splitJobStatus[0]:
            print(f"Successfully completed job to split FASTQ file")
            projectGene = "_".join(jobName.split("_")[0:2])
            filePattern1 = f'{projectGene}_raw.R1.part_*.fastq.gz'
            filePattern2 = f'{projectGene}_raw.R2.part_*.fastq.gz'
            splitFq1 = sorted(glob(os.path.join(outdir, filePattern1)))
            splitFq2 = sorted(glob(os.path.join(outdir, filePattern2)))
            return splitFq1, splitFq2
        else:
            print(f"Failure in job to split FASTQ file")
        return splitFq1, splitFq2

In [20]:
def mergeBatchBamFiles(bamBatchDict: dict, geneConfigs: configUtils.GeneConfig):
    """
    # NOT TO BE USED: MERGING AFFECT REFENAMES
    Merge batches of BAM files
    bamBatchDict: Batch names are keys and values are lists of BAM files for all isoforms.
    Also submit jobs to extract reads names per batch
    """
    mergeBamJobDict = {}
    getReadsIDsJobDict = {}
    mergedBams = []
    seqkit = "/research/rgs01/applications/hpcf/authorized_apps/rhel7_apps/seqkit/vendor/2.1.0/seqkit"
    for batch in bamBatchDict.keys():
        jobName = f'{geneConfigs.project}_{geneConfigs.gene}_merged_{batch}.bam'
        
        outBam = os.path.join(geneConfigs.WDIR, jobName)
        mergedBams.append(outBam)
        batchBams = ' '.join(bamBatchDict[batch])
        mergeCmd = f"{geneConfigs.apps['sambamba']} merge {outBam} {batchBams}"
        indexCmd =  f"{geneConfigs.apps['sambamba']} index {outBam}"
        cmd = f'"{mergeCmd} && {indexCmd}"'
        mergeBamJobDict[jobName] = cmd
        
        jobNameRead = f'{geneConfigs.project}_{geneConfigs.gene}_{batch}_readnames'
        outReadFn = os.path.join(geneConfigs.WDIR, jobNameRead)
        extractReadIn = os.path.join(geneConfigs.WDIR, f'{geneConfigs.project}_{geneConfigs.gene}_raw.R1.{batch}.fastq.gz')
        readExtractCmd = f'"{seqkit} fx2tab -n {extractReadIn} > {outReadFn}"'
        getReadsIDsJobDict[jobNameRead] = readExtractCmd
    
    mergeJobDict = ngsUtils.submitJobs(mergeBamJobDict, wdir=geneConfigs.WDIR)
    print(f"Submitted {len(mergeJobDict)} jobs to merge isoforms specific BAMs per batch")
    readNamesJobDict = ngsUtils.submitJobs(getReadsIDsJobDict, wdir=geneConfigs.WDIR)
    print(f"Submitted {len(readNamesJobDict)} jobs to extract readnames per batch")
    mergeJobStatus = ngsUtils.aa08_job_manager_succeeded(
        mergeJobDict, only_check_finish=False, initial_sleep_sec=10, wait_sec=5, outpath=geneConfigs.WDIR
    )
    if mergeJobStatus[0]:
        print(f"Successfully completed {len(mergeJobDict)} jobs to merge isoforms specific BAMs per batch")
        return mergedBams, readNamesJobDict
    else:
        print(f"Failure in jobs to merge isoforms specific BAMs per batch")
    return mergedBams, readNamesJobDict

In [21]:
def mapFastqReadsMultiplePairs(
        bwaIndices: list,
        fq1: list,
        fq2: list,
        outdir: os.path,
        threads: int = 4,

):
    mappingScript = "/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/rnaseq/rnaseq/mapReads.py"
    mappingJobDict = {}
    for i in range(len(fq1)):
        r1 = fq1[i]
        r2 = fq2[i]
        for index in bwaIndices:
            batchPrefix = os.path.basename(r1).replace(".R1.", ".").replace(".fastq.gz", "")
            isoform = "_".join(os.path.basename(index).replace(".fa", "").split("_")[2:])
            prefix = batchPrefix + "_isoform_" + isoform
            mapCmd = f"python {mappingScript} -index {index} -fq1 {r1} -fq2 {r2} -prefix {prefix} -threads {threads}"
            mappingJobDict[prefix] = mapCmd
    submitJobDict = submitJobs(mappingJobDict, wdir=outdir)
    mapJobStatus = ngsUtils.aa08_job_manager_succeeded(submitJobDict, only_check_finish=True, initial_sleep_sec=5, wait_sec=3, outpath=outdir)
    print(f"Submiited {len(mappingJobDict)} jobs for mapping raw batches of reads")
    if mapJobStatus[0]:
          print(f"Successfully completed {len(mappingJobDict)} jobs for mapping raw batches of reads")
    else:
          print(f"Failure in {len(mappingJobDict)} jobs for mapping raw batches of reads")
    return mapJobStatus

In [22]:
def extractFastqsFromSampleBatches(
      sampleDf: pd.DataFrame,
      geneConfigs: configUtils.GeneConfig,
      batches: int,
      geneConfigPickleFile: os.path,
):
      totalSamples = 0
      batchSizes = []
      #batchDfs = []
      batchNums = 1
      batchFastqR1List = []
      batchFastqR2List = []
      batchfiles = []
      fqExtractionBatchCmdsDict = {}
      for batch in np.array_split(sampleDf, batches):
          #batchDfs.append(batch)
          batchSize = batch.shape[0]
          batchSizes.append(batchSize)
          totalSamples += batchSize
          batchFq1 = os.path.join(geneConfigs.WDIR, f"{geneConfigs.project}_{geneConfigs.gene}_batch_{batchNums}.R1.fastq.gz")
          batchFq2 = os.path.join(geneConfigs.WDIR, f"{geneConfigs.project}_{geneConfigs.gene}_batch_{batchNums}.R2.fastq.gz")
          batchFastqR1List.append(batchFq1)
          batchFastqR2List.append(batchFq2)
          batchofn = os.path.join(geneConfigs.WDIR, f"{geneConfigs.project}_fastqExtraction_{batchNums}.tsv")
          batch.to_csv(batchofn, sep="\t", index=False)
          batchfiles.append(batchofn)
          script = "/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/rnaseq/rnaseq/extractGeneFastqs.py"
          batchCmd = f"python {script} -samplefile {batchofn} -batchNum {batchNums} -pickelfile {geneConfigPickleFile}"
          jobName = f"{geneConfigs.project}_{geneConfigs.gene}_batch_{batchNums}"
          fqExtractionBatchCmdsDict[jobName] = batchCmd
          batchNums += 1
      print(f"Created {batches} batches from {totalSamples} samples")
      fastqSubmissionJobDict = ngsUtils.submitJobs(fqExtractionBatchCmdsDict, queue='compbio', memory='8000', wdir=geneConfigs.WDIR)
      print(f"Submiited {len(fqExtractionBatchCmdsDict)} jobs for extraction of FASTQ reads")
      fastqJobStatus = ngsUtils.aa08_job_manager_succeeded(
            fastqSubmissionJobDict, only_check_finish=False, initial_sleep_sec=10, wait_sec=5, outpath=geneConfigs.WDIR
      )
      if fastqJobStatus[0]:
            print(f"Successfully completed {len(fqExtractionBatchCmdsDict)} jobs for extraction of FASTQ reads")
      else:
            print(f"Failure {len(fqExtractionBatchCmdsDict)} jobs for extraction of FASTQ reads")
      return fastqJobStatus

In [23]:
def processFastqExtractionBatch(batchDf: pd.DataFrame, batchNum: int, geneConfigs: configUtils.GeneConfig) -> dict:
    for index, row in batchDf.iterrows():
        sampleName = row['SID']
        bam = row['BAM']
        suffix = f"prebatch_{batchNum}"
        fastqfiles = extractGeneReadsFromBAM(bam, sampleName, geneConfigs.WDIR, geneConfigs, suffix=suffix)
    r1Pattern = f'"{geneConfigs.project}_{geneConfigs.gene}*{suffix}.gene.*.R1.fastq.gz"'
    r2Pattern = f'"{geneConfigs.project}_{geneConfigs.gene}*{suffix}.gene.*.R2.fastq.gz"'
    batchFq1 = os.path.join(geneConfigs.WDIR, f"{geneConfigs.project}_{geneConfigs.gene}_batch_{batchNum}.R1.fastq.gz")
    batchFq2 = os.path.join(geneConfigs.WDIR, f"{geneConfigs.project}_{geneConfigs.gene}_batch_{batchNum}.R2.fastq.gz")
    return concatFastqBatches(geneConfigs.WDIR, r1Pattern, r2Pattern, batchFq1, batchFq2)

In [24]:
def saveGeneConfig(configObject, fname):
    with open(fname, 'wb') as handle:
        pickle.dump(configObject, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return fname

def loadConfig(pickleFile):
    with open(pickleFile, 'rb') as handle:
        configObject = pickle.load(handle)
    return configObject

## Define global and gene specific configurations

#### Configuration

In [131]:
projectid = "TARGETAML"
gene = "NRAS"
batches = 2
configFile = "/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/rnaseq/config.ini"
sampleFile = "/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/rnaseq/data/samples.tsv"
BWAReadsPerBatch = 10000
splitFqThreads = 1
mappingGenome = 'hg38'

In [132]:
geneConfigs = configUtils.GeneConfig(configFile=configFile, gene=gene, projectID=projectid)
#pickleFname = f"{geneConfigs.WDIR}/{geneConfigs.project}_{geneConfigs.gene}.pickle"

[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.00 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.12-r1039
[main] CMD: /research/groups/xmagrp/projects/SensitiveDetection/xmagrp/code/dipseek/apps/bwa index /research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/NRAS_hg19_NM_002524.fa
[main] Real time: 0.009 sec; CPU: 0.020 sec
[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.00 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.12-r1039
[main] CMD: /research/groups/xmagrp/projects/SensitiveDetection/xmagrp/code/dipseek/apps/bwa index /research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp

In [130]:
geneConfigs.configPickle

'/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS.pickle'

In [129]:
geneConfigs.confDict['PARAMETERS']['splitfqthreads']

'1'

In [133]:
geneConfigs.confDict['PARAMETERS']['readsbatchsize']

'10000'

In [27]:
geneConfigs.confDict['hg19']['txnFeatures']

['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/NRAS_hg19_NM_002524_features.tsv']

In [156]:
geneConfigs.confDict['PARAMETERS']['mapgenome'], geneConfigs.confDict['PARAMETERS']['trimlen']

('hg38', '5')

In [46]:
geneConfigs.configPickle

'/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS.pickle'

In [47]:
pickleFile = geneConfigs.configPickle

In [122]:
sampleDf = prepareSampleNameMap(sampleFile, geneConfigs.WDIR, geneConfigs.project)

In [32]:
geneConfigs.confDict['hg19']['Isoforms']

['NM_002524']

In [34]:
vars(geneConfigs)

{'configFile': '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/rnaseq/config.ini',
 'gene': 'NRAS',
 'project': 'TARGETAML',
 'confDict': {'hg19': {'bwaIndices': ['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/NRAS_hg19_NM_002524.fa',
    '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/NRAS_hg19_SUPERRNA.fa',
    '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/NRAS_hg19_SUPERDNA.fa'],
   'isoforms':        bin       nmID chrom strand        txS        txE       cdsS   
   3121  1464  NM_002524  chr1      -  115247089  115259392  115251155  \
   
              cdsE  exonN                                              exonS   
   3121  115258781      7  115247089,115250774,115251151,115252189,115256...  \
   
         ... gName  cdsStartStat cdsEndStat         exonFrames isInjected isPAR   
   3121  ...  NRAS

In [28]:
geneConfigs.apps['seqkit']

'/research/groups/xmagrp/projects/SensitiveDetection/xmagrp/code/dipseek/apps/seqkit'

In [27]:
geneConfigs.confDict['hg19']['bwaIndices']

['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/NRAS_hg19_NM_002524.fa',
 '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/NRAS_hg19_SUPERRNA.fa',
 '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/NRAS_hg19_SUPERDNA.fa']

In [32]:
geneConfigs.confDict['PARAMETERS']

{'mapq': '55', 'mapgenome': 'hg38'}

### Extract FASTQ reads in batches 

In [123]:
sampleDf.head(6)

,Sample,BAM,SID
0,817151_TTAGGC_L001_concat,/research/groups/xmagrp/projects/AMLRelapse/co...,S1
1,835165_ACTTGA_L001_concat,/research/groups/xmagrp/projects/AMLRelapse/co...,S2
2,BM3897-09A-01R_withJunctionsOnGenome_dupsFlagged,/research/groups/xmagrp/projects/AMLRelapse/co...,S3
3,BM3969-09A-01R_withJunctionsOnGenome_dupsFlagged,/research/groups/xmagrp/projects/AMLRelapse/co...,S4
4,BM4203-09A-01R_withJunctionsOnGenome_dupsFlagged,/research/groups/xmagrp/projects/AMLRelapse/co...,S5
5,BM4404-09A-01R_withJunctionsOnGenome_dupsFlagged,/research/groups/xmagrp/projects/AMLRelapse/co...,S6


In [125]:
list(sampleDf['BAM'].head())

['/research/groups/xmagrp/projects/AMLRelapse/common/Meshinchi/TARGET_AML/bam/817151_TTAGGC_L001_concat.bam',
 '/research/groups/xmagrp/projects/AMLRelapse/common/Meshinchi/TARGET_AML/bam/835165_ACTTGA_L001_concat.bam',
 '/research/groups/xmagrp/projects/AMLRelapse/common/Meshinchi/TARGET_AML/bam/BM3897-09A-01R_withJunctionsOnGenome_dupsFlagged.bam',
 '/research/groups/xmagrp/projects/AMLRelapse/common/Meshinchi/TARGET_AML/bam/BM3969-09A-01R_withJunctionsOnGenome_dupsFlagged.bam',
 '/research/groups/xmagrp/projects/AMLRelapse/common/Meshinchi/TARGET_AML/bam/BM4203-09A-01R_withJunctionsOnGenome_dupsFlagged.bam']

In [32]:
fastqExtractionJobStatus = extractFastqsFromSampleBatches(
      sampleDf=sampleDf.head(6),
      geneConfigs=geneConfigs,
      batches=batches,
      geneConfigPickleFile=pickleFile
)

Created 2 batches from 6 samples
Job <227807466> is submitted to queue <compbio>.
Job <227807469> is submitted to queue <compbio>.
Submiited 2 jobs for extraction of FASTQ reads
job checking round: 1; finish: 0/2
job checking round: 2; finish: 0/2
job checking round: 3; finish: 2/2
Successfully completed 2 jobs for extraction of FASTQ reads


### Concat FASTQs from sample batches

In [33]:
catFastqs = concatFastqBatches(
    outdir=geneConfigs.WDIR,
    r1Pattern=f'"{geneConfigs.project}_{geneConfigs.gene}_batch_*.R1.fastq.gz"',
    r2Pattern=f'"{geneConfigs.project}_{geneConfigs.gene}_batch_*.R2.fastq.gz"',
    fq1=os.path.join(geneConfigs.WDIR, f"{geneConfigs.project}_{geneConfigs.gene}_raw.R1.fastq.gz"),
    fq2=os.path.join(geneConfigs.WDIR, f"{geneConfigs.project}_{geneConfigs.gene}_raw.R2.fastq.gz")
)

In [34]:
catFastqs

['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_raw.R1.fastq.gz',
 '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_raw.R2.fastq.gz']

### Split FASTQs in set batch sizes

In [35]:
splitFqJobName = f"{geneConfigs.project}_{geneConfigs.gene}_SPLIT_FASTQ"

splitFq1, splitFq2 = splitFastq(
    fq1=catFastqs[0], fq2=catFastqs[1],
    batchSize=BWAReadsPerBatch, outdir=geneConfigs.WDIR, threads=splitFqThreads, jobName=splitFqJobName,
    seqkit=geneConfigs.apps['seqkit']
)

Job <227807593> is submitted to queue <compbio>.
Submitted a job to split FASTQ file
job checking round: 1; finish: 1/1
Successfully completed job to split FASTQ file


### Extract all readnames per batch

In [36]:
readNameFiles, batchIDs = extractReadNamesFromFASTQ(splitFq1, geneConfigs)

Job <227807598> is submitted to queue <compbio>.
Job <227807599> is submitted to queue <compbio>.
Job <227807600> is submitted to queue <compbio>.
Submitted 3 jobs to Extract readnames from the FASTQ batches

job checking round: 1; finish: 3/3
Completed 3 jobs to Extract readnames from the FASTQ batches


In [37]:
batchIDs

['part_001', 'part_002', 'part_003']

In [38]:
readNameFiles

['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_part_001_readnames',
 '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_part_002_readnames',
 '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_part_003_readnames']

### Align each batch of FASTQ in priority order of the isoforms and super-RNA and -DNA of the gene

In [39]:
geneConfigs.confDict['hg38']['bwaIndices']

['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/NRAS_hg38_NM_002524.fa',
 '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/NRAS_hg38_SUPERRNA.fa',
 '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/NRAS_hg38_SUPERDNA.fa']

In [111]:
batchJobStatus, completeBams, matchedBams = batchWisePerfectMatchedBAMs(splitFq1=splitFq1, splitFq2=splitFq2, geneConfigs=geneConfigs)

Job <227820844> is submitted to queue <compbio>.
Job <227820845> is submitted to queue <compbio>.
Job <227820846> is submitted to queue <compbio>.
Submitted 3 jobs to Map FASTQ Batches in order of isoforms to subset perfectly matched reads for counting

job checking round: 1; finish: 0/3
job checking round: 2; finish: 0/3
job checking round: 3; finish: 0/3
job checking round: 4; finish: 0/3
job checking round: 5; finish: 0/3
job checking round: 6; finish: 0/3
job checking round: 7; finish: 0/3
job checking round: 8; finish: 0/3
job checking round: 9; finish: 0/3
job checking round: 10; finish: 0/3
job checking round: 11; finish: 0/3
job checking round: 12; finish: 0/3
job checking round: 13; finish: 0/3
job checking round: 14; finish: 0/3
job checking round: 15; finish: 3/3
Completed 3 jobs to Map FASTQ Batches in order of isoforms to subset perfectly matched reads for counting


In [141]:
matchedBams

{'NM_002524': ['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524_part_001_MATCHEDREADS_0.bam',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524_part_002_MATCHEDREADS_0.bam',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524_part_003_MATCHEDREADS_0.bam'],
 'SUPERRNA': ['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_SUPERRNA_part_001_MATCHEDREADS_1.bam',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_SUPERRNA_part_002_MATCHEDREADS_1.bam',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_SUPERRNA_part_003_MATCHEDREADS_1.bam'],
 'SUPERDNA': ['/research_jude/rgs01_jude/groups/xmagrp/pro

In [42]:
completeBams

{'NM_002524': ['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524_part_001.bam',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524_part_002.bam',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524_part_003.bam'],
 'SUPERRNA': ['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_SUPERRNA_part_001.bam',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_SUPERRNA_part_002.bam',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_SUPERRNA_part_003.bam'],
 'SUPERDNA': ['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_SUPERDNA_part_001.bam',

### Counting allles for SNV calling

In [50]:
geneConfigs.confDict['PARAMETERS']['mapq']

'15'

In [158]:
countJobStatus, countFiles, errFiles = alleleCountingForPerfectMatchedBAMs(matchedBams=matchedBams, geneConfigs=geneConfigs)

Job <227837866> is submitted to queue <compbio>.
Job <227837867> is submitted to queue <compbio>.
Job <227837869> is submitted to queue <compbio>.
Job <227837870> is submitted to queue <compbio>.
Job <227837872> is submitted to queue <compbio>.
Job <227837874> is submitted to queue <compbio>.
Job <227837875> is submitted to queue <compbio>.
Job <227837877> is submitted to queue <compbio>.
Job <227837878> is submitted to queue <compbio>.
Submitted 9 jobs to Counting alleles in perfectly matched BAMs in batch-wise manner

job checking round: 1; finish: 2/9
job checking round: 2; finish: 5/9
job checking round: 3; finish: 5/9
job checking round: 4; finish: 7/9
job checking round: 5; finish: 7/9
job checking round: 6; finish: 7/9
job checking round: 7; finish: 7/9
job checking round: 8; finish: 7/9
job checking round: 9; finish: 8/9
job checking round: 10; finish: 8/9
job checking round: 11; finish: 8/9
job checking round: 12; finish: 9/9
Completed 9 jobs to Counting alleles in perfectly m

In [159]:
countFiles

{'NM_002524': ['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524_part_001_MATCHEDREADS_0.count',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524_part_002_MATCHEDREADS_0.count',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524_part_003_MATCHEDREADS_0.count'],
 'SUPERRNA': ['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_SUPERRNA_part_001_MATCHEDREADS_1.count',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_SUPERRNA_part_002_MATCHEDREADS_1.count',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_SUPERRNA_part_003_MATCHEDREADS_1.count'],
 'SUPERDNA': ['/research_jude/rgs01_jude/group

#### Aggregate counts for isoform batches

In [114]:
def groupbySumDfs(df1: pd.DataFrame, df2: pd.DataFrame):
    if 'Sample' in list(df1.columns):
        groupCols = ['Sample', 'Chr', 'Pos']
    if 'Tile' in list(df1.columns):
        groupCols = ['Tile']
    if 'Junction' in list(df1.columns):
        groupCols = ['Sample', 'Chr', 'Junction']
    return pd.concat([df1, df2]).groupby(groupCols).agg('sum').reset_index()

def aggregateCounts(files: list, groupCols: list = ['Sample', 'Chr', 'Pos'], qcut: int = 30):
    from functools import reduce
    dfs = [pd.read_csv(fname, sep="\t", low_memory=False) for fname in files]
    bases = ['A', 'C', 'G', 'T']
    if groupCols == ['Sample', 'Chr', 'Pos']:
        retCols = groupCols + [f'{base}_Q_{qcut}' for base in bases] + [f'N_Q_{qcut}']
        initDf = pd.DataFrame(columns=retCols)
        dfs = [initDf] + dfs
    elif groupCols == ['Tile']:
        dimers = []
        for base1 in bases:
            for base2 in bases:
                dimer = base1+base2
                dimers.append(dimer)
        retCols = ['Tile'] + dimers
        initDf = pd.DataFrame(columns=retCols)
        dfs = [initDf] + dfs
    elif groupCols == ['Sample', 'Chr', 'Junction']:
        retCols = groupCols + ['Count']
        initDf = pd.DataFrame(columns=retCols)
        dfs = [initDf] + dfs

    return reduce(groupbySumDfs, dfs).sort_values(by=groupCols).reset_index(drop=True)

def aggregateIsoformCounts(
        countFilesDict: dict, geneConfigs: configUtils.GeneConfig, 
        groupCols: list = ['Sample', 'Chr', 'Pos']
):
    aggCountFilesDict = {}
    if 'Sample' in groupCols:
        extension = ".count"
    if 'Tile' in groupCols:
        extension = ".count_err"
    for isoform in countFilesDict.keys():
        isofn = os.path.join(
            geneConfigs.WDIR, geneConfigs.project + '_' + geneConfigs.gene + '_' + isoform + extension
        )
        isoCountDf = aggregateCounts(files=countFilesDict[isoform], groupCols=groupCols)
        isoCountDf.to_csv(isofn, sep="\t", index=False)
        aggCountFilesDict[isoform] = isofn
    return aggCountFilesDict

In [98]:
countFiles['NM_002524']

['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524_part_001_MATCHEDREADS_0.count',
 '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524_part_002_MATCHEDREADS_0.count',
 '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524_part_003_MATCHEDREADS_0.count']

In [160]:
aggCountFilesDict = aggregateIsoformCounts(countFilesDict=countFiles, geneConfigs=geneConfigs)

In [161]:
aggCountFilesDict

{'NM_002524': '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524.count',
 'SUPERRNA': '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_SUPERRNA.count',
 'SUPERDNA': '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_SUPERDNA.count'}

### Counting junctions of a given transcript

In [89]:
geneConfigs.confDict['hg38']['txnJunctions']

['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/NRAS_hg38_NM_002524_junctions.tsv']

In [91]:
matchedBams = {'NM_002524': ['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524_part_001_MATCHEDREADS_0.bam',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524_part_002_MATCHEDREADS_0.bam',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524_part_003_MATCHEDREADS_0.bam'],
 'SUPERRNA': ['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_SUPERRNA_part_001_MATCHEDREADS_1.bam',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_SUPERRNA_part_002_MATCHEDREADS_1.bam',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_SUPERRNA_part_003_MATCHEDREADS_1.bam'],
 'SUPERDNA': ['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_SUPERDNA_part_001_MATCHEDREADS_2.bam',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_SUPERDNA_part_002_MATCHEDREADS_2.bam',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_SUPERDNA_part_003_MATCHEDREADS_2.bam']}

In [92]:
matchedBams

{'NM_002524': ['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524_part_001_MATCHEDREADS_0.bam',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524_part_002_MATCHEDREADS_0.bam',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524_part_003_MATCHEDREADS_0.bam'],
 'SUPERRNA': ['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_SUPERRNA_part_001_MATCHEDREADS_1.bam',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_SUPERRNA_part_002_MATCHEDREADS_1.bam',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_SUPERRNA_part_003_MATCHEDREADS_1.bam'],
 'SUPERDNA': ['/research_jude/rgs01_jude/groups/xmagrp/pro

In [95]:
targetGenome = geneConfigs.confDict['PARAMETERS']['mapgenome']

In [99]:
isoforms = geneConfigs.confDict[targetGenome]['Isoforms']

In [100]:
geneConfigs.confDict[targetGenome]['txnJunctions']

['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/NRAS_hg38_NM_002524_junctions.tsv']

In [117]:
def countJunctionsOfTranscripts(
        matchedBamsDict: dict, geneConfigs: configUtils.GeneConfig
):
    targetGenome = geneConfigs.confDict['PARAMETERS']['mapgenome']
    isoforms = geneConfigs.confDict[targetGenome]['Isoforms']
    junctionsJobDict = {}
    for isoform in isoforms:
        bams = matchedBamsDict[isoform]
        junctionfile = None
        script = "/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/rnaseq/rnaseq/junctionCounter.py"
        for jnfn in geneConfigs.confDict[targetGenome]['txnJunctions']:
            if isoform in os.path.basename(jnfn):
                junctionfile = jnfn
        for bam in bams:
            jobName = os.path.basename(bam).replace('.bam', '') + '_junctions.count'
            ofn = os.path.join(
                geneConfigs.WDIR,
                jobName
            )
            junctionsJobDict[jobName] = f"python {script} -bam {bam} -junctionfile {junctionfile} -ofn {ofn}"
    
    junctionCountJobStatus = ngsUtils.submitAndMonitorJobs(
        jobDict=junctionsJobDict, wdir=geneConfigs.WDIR, 
        jobDescription="Counting junctions in perfectly matched BAMs in batch-wise manner"
    )

    jofn = os.path.join(geneConfigs.WDIR, f"{geneConfigs.project}_{geneConfigs.gene}_{targetGenome}_junction.count")
    if junctionCountJobStatus[0]:
        junctionCountFiles = [os.path.join(geneConfigs.WDIR, fileName) for fileName in junctionsJobDict.keys()]
        junctionCountDf = aggregateCounts(files=junctionCountFiles, groupCols=['Sample', 'Chr', 'Junction'])
    else:
        junctionCountDf = pd.DataFrame(columns=['Sample', 'Chr', 'Junction', 'Count'])
    junctionCountDf.to_csv(jofn, sep="\t", index=False)
    return junctionCountJobStatus, jofn


In [119]:
junctionCountJobStatus, jofn = countJunctionsOfTranscripts(matchedBamsDict=matchedBams, geneConfigs=geneConfigs)

Job <228346254> is submitted to queue <compbio>.
Job <228346255> is submitted to queue <compbio>.
Job <228346256> is submitted to queue <compbio>.
Submitted 3 jobs to Counting junctions in perfectly matched BAMs in batch-wise manner

job checking round: 1; finish: 3/3
Completed 3 jobs to Counting junctions in perfectly matched BAMs in batch-wise manner


In [120]:
jofn

'/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_hg38_junction.count'

In [105]:
junctionsJobDict = {}
for isoform in isoforms:
    bams = matchedBams[isoform]
    junctionfile = None
    script = "/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/rnaseq/rnaseq/junctionCounter.py"
    for jnfn in geneConfigs.confDict[targetGenome]['txnJunctions']:
        if isoform in os.path.basename(jnfn):
            junctionfile = jnfn
    for bam in bams:
        jobName = os.path.basename(bam).replace('.bam', '') + '_junctions.count'
        ofn = os.path.join(
            geneConfigs.WDIR,
            os.path.basename(bam).replace('.bam', '') + '_junctions.count'
        )

        junctionsJobDict[jobName] = f"python {script} -bam {bam} -junctionfile {junctionfile} -ofn {ofn}"

In [106]:
junctionsJobDict

{'TARGETAML_NRAS_NM_002524_part_001_MATCHEDREADS_0_junctions.count': 'python /research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/rnaseq/rnaseq/junctionCounter.py -bam /research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524_part_001_MATCHEDREADS_0.bam -junctionfile /research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/NRAS_hg38_NM_002524_junctions.tsv -ofn /research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524_part_001_MATCHEDREADS_0_junctions.count',
 'TARGETAML_NRAS_NM_002524_part_002_MATCHEDREADS_0_junctions.count': 'python /research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/rnaseq/rnaseq/junctionCounter.py -bam /research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524_part_002_MATCHEDREADS_

### Dropped steps

#### Align each batch of FASTQ against the isoforms and super-DNA and -RNA of the gene

In [25]:
bwaIndices = [geneConfigs.confDict[mappingGenome][isoform] for isoform in geneConfigs.confDict[mappingGenome]['Isoforms'] + ['SUPERRNA', 'SUPERDNA']]

In [128]:
bwaIndices

['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/NRAS_hg38_NM_002524.fa',
 '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/NRAS_hg38_SUPERRNA.fa',
 '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/NRAS_hg38_SUPERDNA.fa']

In [26]:
mapJobStatus = mapFastqReads(
    bwaIndices=bwaIndices, fq1=splitFq1, fq2=splitFq2, outdir=geneConfigs.WDIR
)

Job <225436046> is submitted to queue <rhel8_compbio>.
Job <225436047> is submitted to queue <rhel8_compbio>.
Job <225436048> is submitted to queue <rhel8_compbio>.


Job <225436049> is submitted to queue <rhel8_compbio>.
Job <225436050> is submitted to queue <rhel8_compbio>.
Job <225436051> is submitted to queue <rhel8_compbio>.
Job <225436052> is submitted to queue <rhel8_compbio>.
Job <225436053> is submitted to queue <rhel8_compbio>.
Job <225436054> is submitted to queue <rhel8_compbio>.
job checking round: 1; finish: 1/9
job checking round: 2; finish: 1/9
job checking round: 3; finish: 1/9
job checking round: 4; finish: 1/9
job checking round: 5; finish: 2/9
job checking round: 6; finish: 3/9
job checking round: 7; finish: 3/9
job checking round: 8; finish: 4/9
job checking round: 9; finish: 7/9
job checking round: 10; finish: 8/9
job checking round: 11; finish: 9/9
Submiited 9 jobs for mapping raw batches of reads
Successfully completed 9 jobs for mapping raw batches of reads


In [27]:
mapJobStatus

[True,
 {'TARGETAML_NRAS_raw.part_003_isoform_SUPERDNA': 1,
  'TARGETAML_NRAS_raw.part_003_isoform_SUPERRNA': 1,
  'TARGETAML_NRAS_raw.part_003_isoform_NM_002524': 1,
  'TARGETAML_NRAS_raw.part_002_isoform_SUPERDNA': 1,
  'TARGETAML_NRAS_raw.part_001_isoform_SUPERDNA': 1,
  'TARGETAML_NRAS_raw.part_002_isoform_NM_002524': 1,
  'TARGETAML_NRAS_raw.part_002_isoform_SUPERRNA': 1,
  'TARGETAML_NRAS_raw.part_001_isoform_SUPERRNA': 1,
  'TARGETAML_NRAS_raw.part_001_isoform_NM_002524': 1}]

#### Merge BAMs of all isoforms for each batch

In [130]:
fqBatchesIds

['part_001', 'part_002', 'part_003']

In [51]:
fqBatchesIds = [os.path.basename(fqbatch).split(".")[-3] for fqbatch in splitFq1]
bamBatchesDict = {}
for batch in fqBatchesIds:
    pattern = f'{geneConfigs.WDIR}/{geneConfigs.project}_{geneConfigs.gene}_raw.{batch}_isoform_*.bam'
    bamfiles = glob(pattern)
    bamBatchesDict[batch] = sorted(bamfiles)

In [68]:
mergedBams, readNamesJobDict = mergeBatchBamFiles(bamBatchesDict, geneConfigs)

Job <225518516> is submitted to queue <rhel8_compbio>.
Job <225518517> is submitted to queue <rhel8_compbio>.
Job <225518518> is submitted to queue <rhel8_compbio>.
Submitted 3 jobs to merge isoforms specific BAMs per batch
Job <225518519> is submitted to queue <rhel8_compbio>.
Job <225518520> is submitted to queue <rhel8_compbio>.
Job <225518521> is submitted to queue <rhel8_compbio>.
Submitted 3 jobs to extract readnames per batch
job checking round: 1; finish: 3/3
Successfully completed 3 jobs to merge isoforms specific BAMs per batch


In [111]:
bamBatchesDict

{'part_001': ['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_raw.part_001_isoform_NM_002524.bam',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_raw.part_001_isoform_SUPERDNA.bam',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_raw.part_001_isoform_SUPERRNA.bam'],
 'part_002': ['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_raw.part_002_isoform_NM_002524.bam',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_raw.part_002_isoform_SUPERDNA.bam',
  '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_raw.part_002_isoform_SUPERRNA.bam'],
 'part_003': ['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xma

In [69]:
mergedBams

['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_merged_part_001.bam',
 '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_merged_part_002.bam',
 '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_merged_part_003.bam']

In [70]:
readNamesJobStatus = 

{'TARGETAML_NRAS_part_001_readnames': '"/research/rgs01/applications/hpcf/authorized_apps/rhel7_apps/seqkit/vendor/2.1.0/seqkit fx2tab -n /research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_raw.R1.part_001.fastq.gz > /research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_part_001_readnames"',
 'TARGETAML_NRAS_part_002_readnames': '"/research/rgs01/applications/hpcf/authorized_apps/rhel7_apps/seqkit/vendor/2.1.0/seqkit fx2tab -n /research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_raw.R1.part_002.fastq.gz > /research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_part_002_readnames"',
 'TARGETAML_NRAS_part_003_readnames': '"/research/rgs01/applications/hpcf/authorized_apps/rhel7_apps/seqkit/vendor/2.1.0/seqkit fx2tab -n /research_jude/rgs01_jude/groups/xmagrp/projects

#### Obtain unique FASTQ read IDs per batch

In [71]:
readNamesJobStatus = ngsUtils.aa08_job_manager_succeeded(
    readNamesJobDict, only_check_finish=False, initial_sleep_sec=10, wait_sec=5, outpath=geneConfigs.WDIR
)

job checking round: 1; finish: 3/3


In [72]:
readNamesJobStatus

[True,
 {'TARGETAML_NRAS_part_001_readnames': 1,
  'TARGETAML_NRAS_part_002_readnames': 1,
  'TARGETAML_NRAS_part_003_readnames': 1}]

#### Read classification per batch

In [131]:
list(readNamesJobStatus[1].keys())

['TARGETAML_NRAS_part_001_readnames',
 'TARGETAML_NRAS_part_002_readnames',
 'TARGETAML_NRAS_part_003_readnames']

In [132]:
readNamesBatch = os.path.join(geneConfigs.WDIR, 'TARGETAML_NRAS_part_001_readnames')

In [133]:
readNamesBatch

'/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_part_001_readnames'

In [134]:
batchBam = "/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_merged_part_001.bam"

In [135]:
readnamesdf = pd.read_csv(readNamesBatch, sep="\t", header=None, names=["ReadID"])

In [136]:
readnamesdf

,ReadID
0,HISEQ:1036:HHGJTBCX3:1:1101:3095:19694:S1
1,HISEQ:1036:HHGJTBCX3:1:1101:3362:57492:S1
2,HISEQ:1036:HHGJTBCX3:1:1101:3646:37354:S1
3,HISEQ:1036:HHGJTBCX3:1:1101:3962:8993:S1
4,HISEQ:1036:HHGJTBCX3:1:1101:4199:21460:S1
...,...
9995,HS25_215:5:2205:2963:17455:S3
9996,HS25_215:5:2205:3024:20345:S3
9997,HS25_215:5:2205:3506:8214:S3
9998,HS25_215:5:2205:3826:83639:S3


In [137]:
readid = "HISEQ:1036:HHGJTBCX3:1:1101:3095:19694:S1"

In [138]:
cmd = f"{geneConfigs.apps['sambamba']} view -F \" read_name == '{readid}' \" {batchBam}"

In [139]:
os.system(cmd)

HISEQ:1036:HHGJTBCX3:1:1101:3095:19694:S1	99	NRAS_hg38_NM_002524	758	60	50M	=	849	141	CTGGTCCTGACTTCCCTGGAGGAGAAGTATTCCTGTTGCTGTCTTCAGTC	GGGGGGGIIIIIIIIIIIIGIIGGGIIIIIIIIIIIIIIIIIIIIIIIII	XT:A:U	NM:i:0	SM:i:37	AM:i:37	X0:i:1	X1:i:0	XM:i:0	XO:i:0	XG:i:0	MD:Z:50
HISEQ:1036:HHGJTBCX3:1:1101:3095:19694:S1	147	NRAS_hg38_NM_002524	849	60	50M	=	758	-141	TAGTACAATAATCTCTATTTGAGAAGTTCTCAGAATAACTACCTCCTCAC	IIIIIGIIIIGIIGIIIIIIGIIIIIIIIIIIIIIIIIIIIIIIIGGGGG	XT:A:U	NM:i:0	SM:i:37	AM:i:37	X0:i:1	X1:i:0	XM:i:0	XO:i:0	XG:i:0	MD:Z:50
HISEQ:1036:HHGJTBCX3:1:1101:3095:19694:S1	99	NRAS_hg38_SUPERDNA	8735	60	50M	NRAS_hg38_NM_002524	8826	141	CTGGTCCTGACTTCCCTGGAGGAGAAGTATTCCTGTTGCTGTCTTCAGTC	GGGGGGGIIIIIIIIIIIIGIIGGGIIIIIIIIIIIIIIIIIIIIIIIII	XT:A:U	NM:i:0	SM:i:37	AM:i:37	X0:i:1	X1:i:0	XM:i:0	XO:i:0	XG:i:0	MD:Z:50
HISEQ:1036:HHGJTBCX3:1:1101:3095:19694:S1	147	NRAS_hg38_SUPERDNA	8826	60	50M	NRAS_hg38_NM_002524	8735	-141	TAGTACAATAATCTCTATTTGAGAAGTTCTCAGAATAACTACCTCCTCAC	IIIIIGIIIIGIIGIIIIIIGIIIIIIIIIIIIIIIII

0

In [140]:
p = subprocess.run([cmd], capture_output=True, text=True, shell=True)

In [141]:
pd.DataFrame([ record.split("\t") for record in p.stdout.split("\n")[:-1]])

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,HISEQ:1036:HHGJTBCX3:1:1101:3095:19694:S1,99,NRAS_hg38_NM_002524,758,60,50M,=,849,141,CTGGTCCTGACTTCCCTGGAGGAGAAGTATTCCTGTTGCTGTCTTC...,...,XT:A:U,NM:i:0,SM:i:37,AM:i:37,X0:i:1,X1:i:0,XM:i:0,XO:i:0,XG:i:0,MD:Z:50
1,HISEQ:1036:HHGJTBCX3:1:1101:3095:19694:S1,147,NRAS_hg38_NM_002524,849,60,50M,=,758,-141,TAGTACAATAATCTCTATTTGAGAAGTTCTCAGAATAACTACCTCC...,...,XT:A:U,NM:i:0,SM:i:37,AM:i:37,X0:i:1,X1:i:0,XM:i:0,XO:i:0,XG:i:0,MD:Z:50
2,HISEQ:1036:HHGJTBCX3:1:1101:3095:19694:S1,99,NRAS_hg38_SUPERDNA,8735,60,50M,NRAS_hg38_NM_002524,8826,141,CTGGTCCTGACTTCCCTGGAGGAGAAGTATTCCTGTTGCTGTCTTC...,...,XT:A:U,NM:i:0,SM:i:37,AM:i:37,X0:i:1,X1:i:0,XM:i:0,XO:i:0,XG:i:0,MD:Z:50
3,HISEQ:1036:HHGJTBCX3:1:1101:3095:19694:S1,147,NRAS_hg38_SUPERDNA,8826,60,50M,NRAS_hg38_NM_002524,8735,-141,TAGTACAATAATCTCTATTTGAGAAGTTCTCAGAATAACTACCTCC...,...,XT:A:U,NM:i:0,SM:i:37,AM:i:37,X0:i:1,X1:i:0,XM:i:0,XO:i:0,XG:i:0,MD:Z:50
4,HISEQ:1036:HHGJTBCX3:1:1101:3095:19694:S1,99,NRAS_hg38_SUPERRNA,758,60,50M,NRAS_hg38_NM_002524,849,141,CTGGTCCTGACTTCCCTGGAGGAGAAGTATTCCTGTTGCTGTCTTC...,...,XT:A:U,NM:i:0,SM:i:37,AM:i:37,X0:i:1,X1:i:0,XM:i:0,XO:i:0,XG:i:0,MD:Z:50
5,HISEQ:1036:HHGJTBCX3:1:1101:3095:19694:S1,147,NRAS_hg38_SUPERRNA,849,60,50M,NRAS_hg38_NM_002524,758,-141,TAGTACAATAATCTCTATTTGAGAAGTTCTCAGAATAACTACCTCC...,...,XT:A:U,NM:i:0,SM:i:37,AM:i:37,X0:i:1,X1:i:0,XM:i:0,XO:i:0,XG:i:0,MD:Z:50


In [142]:
p.returncode

0

In [143]:
p = subprocess.run([cmd], capture_output=True, text=True, shell=True)
pd.DataFrame([ record.split("\t") for record in p.stdout.split("\n")[:-1]])
p.returncode

0

In [144]:
extractReadnameRecordFromBAM(readname=readid, bam=batchBam, sambambaPath=geneConfigs.apps['sambamba'])

,qname,flag,rname,pos,mapq,cigar,rnext,pnext,tlen,seq,qual
0,HISEQ:1036:HHGJTBCX3:1:1101:3095:19694:S1,99,NRAS_hg38_NM_002524,758,60,50M,=,849,141,CTGGTCCTGACTTCCCTGGAGGAGAAGTATTCCTGTTGCTGTCTTC...,GGGGGGGIIIIIIIIIIIIGIIGGGIIIIIIIIIIIIIIIIIIIII...
1,HISEQ:1036:HHGJTBCX3:1:1101:3095:19694:S1,147,NRAS_hg38_NM_002524,849,60,50M,=,758,-141,TAGTACAATAATCTCTATTTGAGAAGTTCTCAGAATAACTACCTCC...,IIIIIGIIIIGIIGIIIIIIGIIIIIIIIIIIIIIIIIIIIIIIIG...
2,HISEQ:1036:HHGJTBCX3:1:1101:3095:19694:S1,99,NRAS_hg38_SUPERDNA,8735,60,50M,NRAS_hg38_NM_002524,8826,141,CTGGTCCTGACTTCCCTGGAGGAGAAGTATTCCTGTTGCTGTCTTC...,GGGGGGGIIIIIIIIIIIIGIIGGGIIIIIIIIIIIIIIIIIIIII...
3,HISEQ:1036:HHGJTBCX3:1:1101:3095:19694:S1,147,NRAS_hg38_SUPERDNA,8826,60,50M,NRAS_hg38_NM_002524,8735,-141,TAGTACAATAATCTCTATTTGAGAAGTTCTCAGAATAACTACCTCC...,IIIIIGIIIIGIIGIIIIIIGIIIIIIIIIIIIIIIIIIIIIIIIG...
4,HISEQ:1036:HHGJTBCX3:1:1101:3095:19694:S1,99,NRAS_hg38_SUPERRNA,758,60,50M,NRAS_hg38_NM_002524,849,141,CTGGTCCTGACTTCCCTGGAGGAGAAGTATTCCTGTTGCTGTCTTC...,GGGGGGGIIIIIIIIIIIIGIIGGGIIIIIIIIIIIIIIIIIIIII...
5,HISEQ:1036:HHGJTBCX3:1:1101:3095:19694:S1,147,NRAS_hg38_SUPERRNA,849,60,50M,NRAS_hg38_NM_002524,758,-141,TAGTACAATAATCTCTATTTGAGAAGTTCTCAGAATAACTACCTCC...,IIIIIGIIIIGIIGIIIIIIGIIIIIIIIIIIIIIIIIIIIIIIIG...


In [145]:
extractReadnameRecordFromBAM(readname="HS25_215:5:2205:2963:17455:S3", bam=batchBam, sambambaPath=geneConfigs.apps['sambamba'])

,qname,flag,rname,pos,mapq,cigar,rnext,pnext,tlen,seq,qual
0,HS25_215:5:2205:2963:17455:S3,163,NRAS_hg38_NM_002524,1109,60,75M,=,1250,216,TTCTTTTTACTGTTGAACTTAGAACTATGCTAATTTTTGGAGAAAT...,BCBBCGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGFG...
1,HS25_215:5:2205:2963:17455:S3,83,NRAS_hg38_NM_002524,1250,60,75M,=,1109,-216,TAGATTCATAAATACAAAAATGAATACTGAATTTTGAGTCTATCCT...,GGGGGGFGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...
2,HS25_215:5:2205:2963:17455:S3,163,NRAS_hg38_SUPERDNA,9086,60,75M,NRAS_hg38_NM_002524,9227,216,TTCTTTTTACTGTTGAACTTAGAACTATGCTAATTTTTGGAGAAAT...,BCBBCGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGFG...
3,HS25_215:5:2205:2963:17455:S3,83,NRAS_hg38_SUPERDNA,9227,60,75M,NRAS_hg38_NM_002524,9086,-216,TAGATTCATAAATACAAAAATGAATACTGAATTTTGAGTCTATCCT...,GGGGGGFGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...
4,HS25_215:5:2205:2963:17455:S3,163,NRAS_hg38_SUPERRNA,1109,60,75M,NRAS_hg38_NM_002524,1250,216,TTCTTTTTACTGTTGAACTTAGAACTATGCTAATTTTTGGAGAAAT...,BCBBCGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGFG...
5,HS25_215:5:2205:2963:17455:S3,83,NRAS_hg38_SUPERRNA,1250,60,75M,NRAS_hg38_NM_002524,1109,-216,TAGATTCATAAATACAAAAATGAATACTGAATTTTGAGTCTATCCT...,GGGGGGFGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...


In [146]:
extractReadnameRecordFromIsoformBAMs("HS25_215:5:2205:2963:17455:S3",  bamBatchesDict['part_001'], geneConfigs)

,qname,flag,rname,pos,mapq,cigar,rnext,pnext,tlen,seq,qual
0,HS25_215:5:2205:2963:17455:S3,163,NRAS_hg38_NM_002524,1109,60,75M,=,1250,216,TTCTTTTTACTGTTGAACTTAGAACTATGCTAATTTTTGGAGAAAT...,BCBBCGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGFG...
1,HS25_215:5:2205:2963:17455:S3,83,NRAS_hg38_NM_002524,1250,60,75M,=,1109,-216,TAGATTCATAAATACAAAAATGAATACTGAATTTTGAGTCTATCCT...,GGGGGGFGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...
2,HS25_215:5:2205:2963:17455:S3,163,NRAS_hg38_SUPERDNA,9086,60,75M,=,9227,216,TTCTTTTTACTGTTGAACTTAGAACTATGCTAATTTTTGGAGAAAT...,BCBBCGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGFG...
3,HS25_215:5:2205:2963:17455:S3,83,NRAS_hg38_SUPERDNA,9227,60,75M,=,9086,-216,TAGATTCATAAATACAAAAATGAATACTGAATTTTGAGTCTATCCT...,GGGGGGFGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...
4,HS25_215:5:2205:2963:17455:S3,163,NRAS_hg38_SUPERRNA,1109,60,75M,=,1250,216,TTCTTTTTACTGTTGAACTTAGAACTATGCTAATTTTTGGAGAAAT...,BCBBCGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGFG...
5,HS25_215:5:2205:2963:17455:S3,83,NRAS_hg38_SUPERRNA,1250,60,75M,=,1109,-216,TAGATTCATAAATACAAAAATGAATACTGAATTTTGAGTCTATCCT...,GGGGGGFGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...


In [147]:
bamBatchesDict['part_001']

['/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_raw.part_001_isoform_NM_002524.bam',
 '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_raw.part_001_isoform_SUPERDNA.bam',
 '/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_raw.part_001_isoform_SUPERRNA.bam']

#### Generate batches for parallel execution

In [148]:
tot = 0
batchSizes = []
batchDfs = []
batchNums = 1
batchFastqR1List = []
batchFastqR2List = []
batchfiles = []
fqExtractionBatchCmdsDict = {}
for batch in np.array_split(sampleDf.head(n=6), batches):
    batchDfs.append(batch)
    batchSize = batch.shape[0]
    batchSizes.append(batchSize)
    tot += batchSize
    batchFq1 = os.path.join(geneConfigs.WDIR, f"{geneConfigs.project}_{geneConfigs.gene}_batch_{batchNums}.R1.fastq.gz")
    batchFq2 = os.path.join(geneConfigs.WDIR, f"{geneConfigs.project}_{geneConfigs.gene}_batch_{batchNums}.R2.fastq.gz")
    batchFastqR1List.append(batchFq1)
    batchFastqR2List.append(batchFq2)
    batchofn = os.path.join(geneConfigs.WDIR, f"{geneConfigs.project}_fastqExtraction_{batchNums}.tsv")
    batch.to_csv(batchofn, sep="\t", index=False)                      
    script = "/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/rnaseq/rnaseq/extractGeneFastqs.py"
    batchCmd = f"python {script} -samplefile {batchofn} -batchNum {batchNums} -pickelfile {pickleFile}"
    jobName = f"{geneConfigs.project}_{geneConfigs.gene}_batch_{batchNums}"
    fqExtractionBatchCmdsDict[jobName] = batchCmd
    batchNums += 1
print(tot)

6


In [149]:
len(fqExtractionBatchCmdsDict)
fastqSubmissionJobDict = submitJobs(fqExtractionBatchCmdsDict, 'compbio', '8000', geneConfigs.WDIR)

MEMLIMIT value </research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS> is not valid. Limit must be a positive integer. You can use the following units for the limit: KB (or K), MB (or M), GB (or G), TB (or T), PB (or P), EB (or E), ZB (or Z). Job not submitted.
MEMLIMIT value </research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS> is not valid. Limit must be a positive integer. You can use the following units for the limit: KB (or K), MB (or M), GB (or G), TB (or T), PB (or P), EB (or E), ZB (or Z). Job not submitted.


In [150]:
fastqJobStatus = ngsUtils.aa08_job_manager_succeeded(
    fastqSubmissionJobDict, only_check_finish=False, initial_sleep_sec=10, wait_sec=5, outpath=geneConfigs.WDIR
)

job checking round: 1; finish: 2/2


In [68]:
fastqJobStatus

[True, {'TARGETAML_NRAS_batch_2': 1, 'TARGETAML_NRAS_batch_1': 1}]

## Test/Dev

### Hints for extracting perfect matches and store rest of the read IDs

- Extract reads with given mapping quality and complete match ^\d+M$
    - `sambamba view  -F " mapping_quality >= 30 and cigar =~ /^\d+M$/ " TARGETAML_NRAS_raw.part_001_isoform_NM_002524.bam`
- Extract rest of the reads IDs
    - `sambamba view  -F " not ( mapping_quality >= 30 and not ( unmapped ) and cigar =~ /^\d+M$/ ) " TARGETAML_NRAS_raw.part_001_isoform_NM_002524.bam | cut -f1 | sort | uniq`
- Extract reads with either of the mate is not fully mapped 
    - `sambamba view  -F " mapping_quality >= 30 and not ( unmapped ) and cigar =~ /^\d+M$/ " TARGETAML_NRAS_raw.part_001_isoform_NM_002524.bam | cut -f1 | sort | uniq -c | awk -F '[[:space:]]+' '{if ( $2 == 1 ) print $3}'`


In [372]:
from liftover import get_lifter
def getConverter(source: str = 'hg19', target: str = 'hg38', one_based: bool = True):
    return get_lifter(source, target, one_based=one_based)


def liftoverCoordinates(chrom: str, pos: int, converter:get_lifter):
    pass

In [30]:
hg38fn = "/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/dogma/dev/hg38ProcessedCuratedRefSeq.tsv"
hg19fn = "/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/dogma/dev/hg19ProcessedCuratedRefSeq.tsv"
hg38Space = GeneSpace(refseqfile=hg38fn, genome='hg38')
hg19Space = GeneSpace(refseqfile=hg19fn, genome='hg19')

In [31]:
gata1 = hg38Space.transcriptStructure('NM_002049')

In [32]:
nras = hg38Space.transcriptStructure('NM_002524')

In [33]:
nrasLoci, nrasJunctions = Transcript.transcriptFeatures(nras)

In [34]:
gata1Loci, gata1Junctions = Transcript.transcriptFeatures(gata1)

In [35]:
gata1Junctions

,Gene,Transcript,ExonPair,RNA_pos_A,RNA_pos_B,Chr,DNA_region_A,DNA_region_B,Genome
0,GATA1,NM_002049,E1_E2,52,61,chrX,48786641-48786645,48791091-48791095,hg38
1,GATA1,NM_002049,E2_E3,291,300,chrX,48791325-48791329,48791844-48791848,hg38
2,GATA1,NM_002049,E3_E4,669,678,chrX,48792217-48792221,48792323-48792327,hg38
3,GATA1,NM_002049,E4_E5,815,824,chrX,48792464-48792468,48793172-48793176,hg38
4,GATA1,NM_002049,E5_E6,941,950,chrX,48793293-48793297,48793793-48793797,hg38


In [38]:
for _, row in nrasJunctions.iterrows():
    rnaPosList = set(range(row['RNA_pos_A'], row['RNA_pos_B'] + 1))
    print(row['Transcript'], row['ExonPair'], rnaPosList, len(rnaPosList))

NM_002524 E1_E2 {110, 111, 112, 113, 114, 115, 116, 117, 118, 119} 10
NM_002524 E2_E3 {238, 239, 240, 241, 242, 243, 244, 245, 246, 247} 10
NM_002524 E3_E4 {417, 418, 419, 420, 421, 422, 423, 424, 425, 426} 10
NM_002524 E4_E5 {577, 578, 579, 580, 581, 582, 583, 584, 585, 586} 10
NM_002524 E5_E6 {704, 705, 706, 707, 708, 709, 710, 701, 702, 703} 10
NM_002524 E6_E7 {740, 741, 742, 743, 744, 745, 746, 747, 748, 749} 10


In [115]:
nrasLoci

,Chr,Pos,Feature,FeatureNum,RNAPosForward,RNAPosReversed,Gene,Transcript,Strand,Genome,SeqForward,SeqComplement,Property,AAPos,indexPos,indexSeq,JunctionProperty,JunctionRegion,JunctionFeature
0,chr1,114704469,E,E7,1,4326,NRAS,NM_002524,-,hg38,T,A,3UTR,-1.0,4326.0,A,NaN,NaN,NaN
1,chr1,114704470,E,E7,2,4325,NRAS,NM_002524,-,hg38,T,A,3UTR,-1.0,4325.0,A,NaN,NaN,NaN
2,chr1,114704471,E,E7,3,4324,NRAS,NM_002524,-,hg38,A,T,3UTR,-1.0,4324.0,T,NaN,NaN,NaN
3,chr1,114704472,E,E7,4,4323,NRAS,NM_002524,-,hg38,T,A,3UTR,-1.0,4323.0,A,NaN,NaN,NaN
4,chr1,114704473,E,E7,5,4322,NRAS,NM_002524,-,hg38,G,C,3UTR,-1.0,4322.0,C,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12298,chr1,114716767,E,E1,4322,5,NRAS,NM_002524,-,hg38,G,C,5UTR,-1.0,5.0,C,NaN,NaN,NaN
12299,chr1,114716768,E,E1,4323,4,NRAS,NM_002524,-,hg38,C,G,5UTR,-1.0,4.0,G,NaN,NaN,NaN
12300,chr1,114716769,E,E1,4324,3,NRAS,NM_002524,-,hg38,C,G,5UTR,-1.0,3.0,G,NaN,NaN,NaN
12301,chr1,114716770,E,E1,4325,2,NRAS,NM_002524,-,hg38,C,G,5UTR,-1.0,2.0,G,NaN,NaN,NaN


In [43]:
junctionfile = "/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/NRAS_hg38_NM_002524_junctions.tsv"
bamfile = "/research_jude/rgs01_jude/groups/xmagrp/projects/AMLRelapse/xmagrp/CohortTracker/dev/test/NRAS/TARGETAML_NRAS_NM_002524_part_001_MATCHEDREADS_0.bam"

In [44]:
junctionDf = pd.read_csv(junctionfile, sep="\t")

In [45]:
junctionDf

,Gene,Transcript,ExonPair,RNA_pos_A,RNA_pos_B,Chr,DNA_region_A,DNA_region_B,Genome
0,NRAS,NM_002524,E1_E2,110,119,chr1,114716658-114716662,114716173-114716177,hg38
1,NRAS,NM_002524,E2_E3,238,247,chr1,114716050-114716054,114713974-114713978,hg38
2,NRAS,NM_002524,E3_E4,417,426,chr1,114713800-114713804,114709724-114709728,hg38
3,NRAS,NM_002524,E4_E5,577,586,chr1,114709569-114709573,114708650-114708654,hg38
4,NRAS,NM_002524,E5_E6,701,710,chr1,114708531-114708535,114708188-114708192,hg38
5,NRAS,NM_002524,E6_E7,740,749,chr1,114708154-114708158,114708046-114708050,hg38


In [46]:
bam = pysam.AlignmentFile(filename=bamfile, mode='rb')